# word2vec implementation

## Original papers
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)



## Overview

The original paper proposed using:
1. Matmul to extract word vectors from ```Win```.
2. Softmax to calculate scores with all the word vectors from ```Wout```.

<img src="image/word2vec_cbow_mechanism.png" align="left"/>

## Negative Sampling

### Motivation

The original implementation was computationally expensive.

1. Matmal to extract word vectors from ```Win```.
2. Softmax can happen vocabulary size times with ```Wout```.

<img src="image/word2vec_negative_sampling_motivation.png" align="left"/>


### Solution

1. Use index to extract word vectors from Win.
2. Instead of calculating softmax with all the word vectors from ```Wout```, sample small number (SL) of negative/false word vectors from ```Wout``` and calculate logistic log loss with each sample. 

<img src="image/wors2vec_neg_sample_backprop.png" align="left"/>

## Using only one Word vector space W

There is no reasoning nor proof why two word vector space are required. In the end, we only use one word vector space, which appears to be ```Win```. 

However, if we use one vector space ```W``` for ```event```, ```context``` and ```negative samples```,then an event vector ```event=W[i]``` in a sentence can be used as a negative sample in another setence. Then the weight ```W[i]``` is updated for both positive and negative labels in the same gradient descent on ```W```. The actual [experiment of using only one vector space](./layer/embedding_single_vector_space.py) ```W``` did not work well.

* [Why do we need 2 matrices for word2vec or GloVe](https://datascience.stackexchange.com/a/94422/68313)


<img src="image/word2vec_why_not_one_W.png" align="left" width=800/>

---
# Setups

In [1]:
import cProfile
import sys
import os
import time
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

Colab gets disconnected within approx 20 min. Hence not suitable for training (or need to upgrade to the pro version).

### Clone github to Google Drive

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/drive/MyDrive/github
    !mkdir -p /content/drive/MyDrive/github
    !git clone https://github.com/oonisim/python-programs.git /content/drive/MyDrive/github


### Clone github to local directory

In [3]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

# Jupyter notebook setups

Auto reolaod causes an error in Jupyter notebooks. Restart the Jupyter kernel for the error:
```TypeError: super(type, obj): obj must be an instance or subtype of type```
See
- https://stackoverflow.com/a/52927102/4281353
- http://thomas-cokelaer.info/blog/2011/09/382/

> The problem resides in the mechanism of reloading modules.
> Reloading a module often changes the internal object in memory which
> makes the isinstance test of super return False.

In [4]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [5]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [6]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

Too large LR generates unusable event vector space.

Uniform weight distribution does not work (Why?)
Weights from the normal distribution sampling with small std (0.01) works (Why?)

In [7]:
USE_PTB = True
USE_TEXT8 = False

CORPUS_FILE = "text8_512" if USE_TEXT8 else "ptb_train"
CORPUS_URL = "https://data.deepai.org/text8.zip" \
    if USE_TEXT8 else f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt' \

TARGET_SIZE = TYPE_INT(1)       # Size of the target event (word)
CONTEXT_SIZE = TYPE_INT(10)     # Size of the context in which the target event occurs.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = TYPE_INT(5)       # Size of the negative samples

VECTOR_SIZE = TYPE_INT(100)     # Number of features in the event vector.
WEIGHT_SCHEME = "normal"
WEIGHT_PARAMS = {
    "std": 0.01
}

LR = TYPE_FLOAT(5)
NUM_SENTENCES = 10

STATE_FILE = "../models/word2vec_%s_E%s_C%s_S%s_W%s_%s_%s_V%s_LR%s_N%s.pkl" % (
    CORPUS_FILE,
    TARGET_SIZE,
    CONTEXT_SIZE,
    SAMPLE_SIZE,
    WEIGHT_SCHEME,
    "std",
    WEIGHT_PARAMS["std"],
    VECTOR_SIZE,
    LR,
    NUM_SENTENCES,
)

MAX_ITERATIONS = 100000

---

# Data
## Corpus

In [8]:
path_to_corpus = f"~/.keras/datasets/{CORPUS_FILE}"
if fileio.Function.is_file(path_to_corpus):
    pass
else:
    # text8, run "cat text8 | xargs -n 512 > text8_512" after download
    path_to_corpus = tf.keras.utils.get_file(
        fname=CORPUS_FILE,
        origin=CORPUS_URL,
        extract=True
    )
corpus = fileio.Function.read_file(path_to_corpus)
print(path_to_corpus)

/home/oonisim/.keras/datasets/ptb_train


In [9]:
examples = corpus.split('\n')[:1]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 


---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [10]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing
)

/home/oonisim/conda/envs/python_programs/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [11]:
word_indexing = EventIndexing(
    name="word_indexing",
    corpus=corpus,
    min_sequence_length=WINDOW_SIZE
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [12]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'aer' 'banknote' 'berlitz' 'calloway' 'centrust' 'cluett' 'fromstein' 'gitano']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('aer', 2)
('banknote', 3)
('berlitz', 4)
('calloway', 5)
('centrust', 6)
('cluett', 7)
('fromstein', 8)
('gitano', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
aer                  : 5.34860e-06
banknote             : 5.34860e-06
berlitz              : 5.34860e-06
calloway             : 5.34860e-06
centrust             : 5.34860e-06
cluett               : 5.34860e-06
fromstein            : 5.34860e-06
gitano               : 5.34860e-06


## Sampling using the probability

Sample events according to their probabilities.

In [13]:
sample = word_indexing.sample(size=5)
print(sample)

['expanding', 'ways', 'day', 'anything', 'of']


## Negative Sampling
Sample events not including those events already sampled.

In [14]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")

negative_indices=[162, 484, 1670, 582, 3050] 
events=['are' 'interests' 'temporarily' 'makers' 'jury']


## Sentence to Sequence

In [15]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    34
asbestos        :    63
fiber           :    86
<unk>           :     1
is              :    42
unusually       :    87
<unk>           :     1
once            :    64
it              :    80
enters          :    88
the             :    34
<unk>           :     1


---
# EventContext

EventContext layer generates ```(event, context)``` pairs from sequences.

In [16]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [17]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [18]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold 
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[37, 62, 44, 63, 64, 65, 66, 67, 68, 69, 70],
       [29, 30, 31, 50, 51, 43, 44, 52, 53,  0,  0]])

## (event, context) pairs

For each word (event) in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [19]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:10]

Event context pairs. Shape (2, 11), Target event size 1, Window size 11.


array([[65, 37, 62, 44, 63, 64, 66, 67, 68, 69, 70],
       [43, 29, 30, 31, 50, 51, 44, 52, 53,  0,  0]], dtype=int32)

### (event, context) pairs in textual words

In [20]:
word_indexing.sequence_to_sentence(event_context_pairs[:10])

[['used',
  'a',
  'form',
  'of',
  'asbestos',
  'once',
  'to',
  'make',
  'kent',
  'cigarette',
  'filters'],
 ['chairman',
  'n',
  'years',
  'old',
  'and',
  'former',
  'of',
  'consolidated',
  'gold',
  '<nil>',
  '<nil>']]

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [21]:
%autoreload 2
from layer import (
    Embedding
)
from optimizer import (
    SGD
)

In [22]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    optimizer=SGD(lr=LR),
    dictionary=word_indexing,
    weight_initialization_scheme=WEIGHT_SCHEME,
    weight_initialization_parameters=WEIGHT_PARAMS
)

### Scores ```np.c_[Ye, Ys]``` from the Embedding

The 0th column is the scores for ```dot(Bc, Be``` for positive labels. The rest are the scores for ```dot(Bc, Ws)``` for negative labels.

In [23]:
scores = embedding.function(event_context_pairs)
print(f"Scores:\n{scores[:5]}\n")
print(f"Scores for dot(Bc, Be): \n{scores[:5, :1]}")

Scores:
[[ 4.69841558e-04  1.67092177e-04 -2.82329624e-04  1.15030198e-04 -2.25383643e-04 -4.19924036e-04]
 [-1.51731292e-04  5.58194588e-04 -1.18213458e-04  1.13727045e-04 -1.78170449e-05  1.23736536e-04]]

Scores for dot(Bc, Be): 
[[ 0.00046984]
 [-0.00015173]]


---
# Batch Normalization

In [24]:
from layer.normalization import (
    BatchNormalization
)

In [25]:
bn = BatchNormalization(
    name="bn",
    num_nodes=1+SAMPLE_SIZE
)

---
# Logistic Log Loss

Train the model to get:
1. BoW of the context event vectors close to the target event vector. Label 1
2. BoW of the context event vectors away from each of the negative sample event vectors Label 0.

This is a binary logistic classification, hence use Logistic Log Loss as the network objective function.

In [26]:
from common.function import (
    sigmoid_cross_entropy_log_loss,
    sigmoid
)
from layer.objective import (
    CrossEntropyLogLoss
)

In [27]:
loss = CrossEntropyLogLoss(
    name="loss",
    num_nodes=1,  # Logistic log loss
    log_loss_function=sigmoid_cross_entropy_log_loss
)

---
# Adapter

The logistic log loss layer expects the input of shape ```(N,M=1)```, however Embedding outputs ```(N,(1+SL)``` where ```SL``` is SAMPLE_SIZE. The ```Adapter``` layer bridges between Embedding and the Loss layers.


In [28]:
from layer.adapter import (
    Adapter
)

In [29]:
adapter_function = embedding.adapt_function_to_logistic_log_loss(loss=loss)
adapter_gradient = embedding.adapt_gradient_to_logistic_log_loss()

adapter: Adapter = Adapter(
    name="adapter",
    num_nodes=1,    # Number of output M=1 
    function=adapter_function,
    gradient=adapter_gradient
)

---
# Word2vec Network

## Construct a sequential network

$ \text {Sentences} \rightarrow EventIndexing \rightarrow EventContext \rightarrow  Embedding \rightarrow Adapter \rightarrow LogisticLogLoss$

In [30]:
from network.sequential import (
    SequentialNetwork
)

In [31]:
network = SequentialNetwork(
    name="word2vec",
    num_nodes=1,
    inference_layers=[
        word_indexing,
        event_context,
        embedding,
        adapter
    ],
    objective_layers=[
        loss
    ]
)

## Run training

### Load the saved model file if exists

In [32]:
if fileio.Function.is_file(STATE_FILE):
    print("Loading model...\nSTATE_FILE: %s" % STATE_FILE)
    state = embedding.load(STATE_FILE)

    fmt="""Model loaded.
    event_size %s
    context_size: %s
    event_vector_size: %s
    """
    print(fmt % (
        state["target_size"], 
        state["context_size"], 
        state["event_vector_size"]
    ))
else:
    print("State file does not exist. Saving the initial model to %s." % STATE_FILE)
    embedding.save(STATE_FILE)

Loading model...
STATE_FILE: ../models/word2vec_ptb_train_E1_C10_Wnormal_std_0.01_V100_LR5.0_S5_N10.pkl
Model loaded.
    event_size 1
    context_size: 10
    event_vector_size: 100
    


### Continue training

In [33]:
def sentences_generator(path_to_file, num_sentences):
    stream = fileio.Function.file_line_stream(path_to_file)
    try:
        while True:
            _lines = fileio.Function.take(num_sentences, stream)
            yield np.array(_lines)
    finally:
        stream.close()

# Sentences for the trainig
source = sentences_generator(
    path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
)

# Restore the state if exists.
state = embedding.load(STATE_FILE)

# Continue training
profiler = cProfile.Profile()
profiler.enable()

total_sentences = 0
epochs = 0

for i in range(MAX_ITERATIONS):
    try:
        sentences = next(source)
        total_sentences += len(sentences)

        start = time.time()
        network.train(X=sentences, T=np.array([0]))

        if i % 100 == 0:
            print(
                f"Batch {i:05d} of {NUM_SENTENCES} sentences: "
                f"Average Loss: {np.mean(network.history):10f} "
                f"Duration {time.time() - start:3f}"
            )
        if i % 1000 == 0:
            # embedding.save(STATE_FILE)
            pass
        
    except fileio.Function.GenearatorHasNoMore as e:
        source.close()
        embedding.save(STATE_FILE)

        # Next epoch
        print(f"epoch {epochs} batches {i:05d} done")
        epochs += 1
        source = sentences_generator(
            path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
        )

    except Exception as e:
        print("Unexpected error:", sys.exc_info()[0])
        source.close()
        raise e

embedding.save(STATE_FILE)

profiler.disable()
profiler.print_stats(sort="cumtime")

Batch 00000 of 10 sentences: Average Loss:   0.377837 Duration 0.253498
Batch 00100 of 10 sentences: Average Loss:   0.300687 Duration 0.143290
Batch 00200 of 10 sentences: Average Loss:   0.293107 Duration 0.067000
Batch 00300 of 10 sentences: Average Loss:   0.287961 Duration 0.170852
Batch 00400 of 10 sentences: Average Loss:   0.283184 Duration 0.160169
Batch 00500 of 10 sentences: Average Loss:   0.281906 Duration 0.162519
Batch 00600 of 10 sentences: Average Loss:   0.282994 Duration 0.127383
Batch 00700 of 10 sentences: Average Loss:   0.281182 Duration 0.176040
Batch 00800 of 10 sentences: Average Loss:   0.280507 Duration 0.165288
Batch 00900 of 10 sentences: Average Loss:   0.280256 Duration 0.140948
Batch 01000 of 10 sentences: Average Loss:   0.279790 Duration 0.166355
Batch 01100 of 10 sentences: Average Loss:   0.279876 Duration 0.146388
Batch 01200 of 10 sentences: Average Loss:   0.279638 Duration 0.137938
Batch 01300 of 10 sentences: Average Loss:   0.278991 Duration 0

Batch 11400 of 10 sentences: Average Loss:   0.267056 Duration 0.247263
Batch 11500 of 10 sentences: Average Loss:   0.267063 Duration 0.123812
Batch 11600 of 10 sentences: Average Loss:   0.266976 Duration 0.128988
Batch 11700 of 10 sentences: Average Loss:   0.266924 Duration 0.100958
Batch 11800 of 10 sentences: Average Loss:   0.266836 Duration 0.123029
Batch 11900 of 10 sentences: Average Loss:   0.266797 Duration 0.141814
Batch 12000 of 10 sentences: Average Loss:   0.266719 Duration 0.183207
Batch 12100 of 10 sentences: Average Loss:   0.266704 Duration 0.151992
Batch 12200 of 10 sentences: Average Loss:   0.266672 Duration 0.063920
Batch 12300 of 10 sentences: Average Loss:   0.266562 Duration 0.089968
Batch 12400 of 10 sentences: Average Loss:   0.266467 Duration 0.113766
Batch 12500 of 10 sentences: Average Loss:   0.266428 Duration 0.120758
Batch 12600 of 10 sentences: Average Loss:   0.266367 Duration 0.233368
epoch 2 batches 12623 done
Batch 12700 of 10 sentences: Average 

Batch 22700 of 10 sentences: Average Loss:   0.262544 Duration 0.134503
Batch 22800 of 10 sentences: Average Loss:   0.262552 Duration 0.212068
Batch 22900 of 10 sentences: Average Loss:   0.262518 Duration 0.155276
Batch 23000 of 10 sentences: Average Loss:   0.262459 Duration 0.159432
Batch 23100 of 10 sentences: Average Loss:   0.262419 Duration 0.219003
Batch 23200 of 10 sentences: Average Loss:   0.262392 Duration 0.156004
Batch 23300 of 10 sentences: Average Loss:   0.262318 Duration 0.106737
Batch 23400 of 10 sentences: Average Loss:   0.262236 Duration 0.096243
Batch 23500 of 10 sentences: Average Loss:   0.262212 Duration 0.141431
Batch 23600 of 10 sentences: Average Loss:   0.262146 Duration 0.211659
Batch 23700 of 10 sentences: Average Loss:   0.262106 Duration 0.142459
Batch 23800 of 10 sentences: Average Loss:   0.262116 Duration 0.197581
Batch 23900 of 10 sentences: Average Loss:   0.262076 Duration 0.154832
Batch 24000 of 10 sentences: Average Loss:   0.262027 Duration 0

Batch 34000 of 10 sentences: Average Loss:   0.258742 Duration 0.260818
Batch 34100 of 10 sentences: Average Loss:   0.258691 Duration 0.110821
Batch 34200 of 10 sentences: Average Loss:   0.258680 Duration 0.185052
Batch 34300 of 10 sentences: Average Loss:   0.258675 Duration 0.139431
Batch 34400 of 10 sentences: Average Loss:   0.258632 Duration 0.114731
Batch 34500 of 10 sentences: Average Loss:   0.258607 Duration 0.142235
Batch 34600 of 10 sentences: Average Loss:   0.258591 Duration 0.164371
Batch 34700 of 10 sentences: Average Loss:   0.258568 Duration 0.133128
Batch 34800 of 10 sentences: Average Loss:   0.258563 Duration 0.117076
Batch 34900 of 10 sentences: Average Loss:   0.258553 Duration 0.125717
Batch 35000 of 10 sentences: Average Loss:   0.258513 Duration 0.141819
Batch 35100 of 10 sentences: Average Loss:   0.258481 Duration 0.236598
Batch 35200 of 10 sentences: Average Loss:   0.258475 Duration 0.116731
Batch 35300 of 10 sentences: Average Loss:   0.258442 Duration 0

Batch 45400 of 10 sentences: Average Loss:   0.255576 Duration 0.163643
Batch 45500 of 10 sentences: Average Loss:   0.255553 Duration 0.152171
Batch 45600 of 10 sentences: Average Loss:   0.255507 Duration 0.165099
Batch 45700 of 10 sentences: Average Loss:   0.255480 Duration 0.169909
Batch 45800 of 10 sentences: Average Loss:   0.255471 Duration 0.133723
Batch 45900 of 10 sentences: Average Loss:   0.255427 Duration 0.176315
Batch 46000 of 10 sentences: Average Loss:   0.255397 Duration 0.136685
Batch 46100 of 10 sentences: Average Loss:   0.255349 Duration 0.135832
Batch 46200 of 10 sentences: Average Loss:   0.255314 Duration 0.114719
epoch 10 batches 46287 done
Batch 46300 of 10 sentences: Average Loss:   0.255285 Duration 0.153073
Batch 46400 of 10 sentences: Average Loss:   0.255280 Duration 0.146082
Batch 46500 of 10 sentences: Average Loss:   0.255257 Duration 0.087204
Batch 46600 of 10 sentences: Average Loss:   0.255227 Duration 0.163894
Batch 46700 of 10 sentences: Average

...

Batch 68000 of 10 sentences: Average Loss:   0.250072 Duration 0.201376
Batch 68100 of 10 sentences: Average Loss:   0.250035 Duration 0.113745
Batch 68200 of 10 sentences: Average Loss:   0.250026 Duration 0.182832
Batch 68300 of 10 sentences: Average Loss:   0.250007 Duration 0.152914
Batch 68400 of 10 sentences: Average Loss:   0.249995 Duration 0.141424
Batch 68500 of 10 sentences: Average Loss:   0.249984 Duration 0.173476
Batch 68600 of 10 sentences: Average Loss:   0.249969 Duration 0.175088
Batch 68700 of 10 sentences: Average Loss:   0.249931 Duration 0.119848
Batch 68800 of 10 sentences: Average Loss:   0.249916 Duration 0.170043
Batch 68900 of 10 sentences: Average Loss:   0.249906 Duration 0.148525
Batch 69000 of 10 sentences: Average Loss:   0.249881 Duration 0.153543
Batch 69100 of 10 sentences: Average Loss:   0.249871 Duration 0.156125
Batch 69200 of 10 sentences: Average Loss:   0.249844 Duration 0.103979
Batch 69300 of 10 sentences: Average Loss:   0.249818 Duration 0

Batch 79400 of 10 sentences: Average Loss:   0.247639 Duration 0.130252
Batch 79500 of 10 sentences: Average Loss:   0.247623 Duration 0.187007
Batch 79600 of 10 sentences: Average Loss:   0.247594 Duration 0.107959
Batch 79700 of 10 sentences: Average Loss:   0.247568 Duration 0.182772
Batch 79800 of 10 sentences: Average Loss:   0.247536 Duration 0.150264
Batch 79900 of 10 sentences: Average Loss:   0.247519 Duration 0.174709
epoch 18 batches 79951 done
Batch 80000 of 10 sentences: Average Loss:   0.247495 Duration 0.219630
Batch 80100 of 10 sentences: Average Loss:   0.247481 Duration 0.147173
Batch 80200 of 10 sentences: Average Loss:   0.247470 Duration 0.196369
Batch 80300 of 10 sentences: Average Loss:   0.247435 Duration 0.248974
Batch 80400 of 10 sentences: Average Loss:   0.247412 Duration 0.256680
Batch 80500 of 10 sentences: Average Loss:   0.247403 Duration 0.146693
Batch 80600 of 10 sentences: Average Loss:   0.247386 Duration 0.152255
Batch 80700 of 10 sentences: Average

Batch 90700 of 10 sentences: Average Loss:   0.245370 Duration 0.256352
Batch 90800 of 10 sentences: Average Loss:   0.245360 Duration 0.170192
Batch 90900 of 10 sentences: Average Loss:   0.245334 Duration 0.141894
Batch 91000 of 10 sentences: Average Loss:   0.245308 Duration 0.165189
Batch 91100 of 10 sentences: Average Loss:   0.245300 Duration 0.176229
Batch 91200 of 10 sentences: Average Loss:   0.245281 Duration 0.135576
Batch 91300 of 10 sentences: Average Loss:   0.245261 Duration 0.138963
Batch 91400 of 10 sentences: Average Loss:   0.245240 Duration 0.160622
Batch 91500 of 10 sentences: Average Loss:   0.245226 Duration 0.165834
Batch 91600 of 10 sentences: Average Loss:   0.245207 Duration 0.175700
Batch 91700 of 10 sentences: Average Loss:   0.245189 Duration 0.196054
Batch 91800 of 10 sentences: Average Loss:   0.245171 Duration 0.145123
Batch 91900 of 10 sentences: Average Loss:   0.245141 Duration 0.153350
Batch 92000 of 10 sentences: Average Loss:   0.245124 Duration 0

   299931    0.795    0.000    7.043    0.000 gen_math_ops.py:7157(_range)
  1499609    4.455    0.000    6.777    0.000 re.py:289(_compile)
   999770    1.351    0.000    6.679    0.000 base.py:126(is_same_shape)
   199954    0.654    0.000    6.531    0.000 composite.py:146(T)
  1399678    6.293    0.000    6.293    0.000 {method 'reshape' of 'numpy.ndarray' objects}
  2099471    2.467    0.000    6.266    0.000 base.py:130(tensor_rank)
 14715076    6.184    0.000    6.184    0.000 embedding.py:201(dictionary)
   999724    0.931    0.000    6.077    0.000 fromnumeric.py:3709(round_)
  1399678    2.139    0.000    6.040    0.000 {built-in method builtins.all}
  1099747    1.598    0.000    5.835    0.000 tensor_util.py:1042(maybe_set_static_shape)
  2999172    2.415    0.000    5.782    0.000 <__array_function__ internals>:2(ndim)
   499885    2.661    0.000    5.698    0.000 arrayprint.py:60(_make_options_dict)
  5798666    5.573    0.000    5.573    0.000 _internal.py:826(npy_ctypes

   299931    0.135    0.000    0.135    0.000 tensor_shape.py:851(__bool__)
     3069    0.017    0.000    0.127    0.000 iostream.py:197(schedule)
    99977    0.124    0.000    0.124    0.000 preprocess.py:104(min_sequence_length)
    14829    0.061    0.000    0.112    0.000 codecs.py:319(decode)
   199954    0.104    0.000    0.104    0.000 multiarray.py:1054(copyto)
   199954    0.094    0.000    0.094    0.000 arraysetops.py:494(_in1d_dispatcher)
   100977    0.093    0.000    0.093    0.000 {built-in method time.time}
     3069    0.086    0.000    0.086    0.000 socket.py:432(send)
   199954    0.077    0.000    0.077    0.000 arraysetops.py:611(_isin_dispatcher)
   199954    0.073    0.000    0.073    0.000 fromnumeric.py:2106(_sum_dispatcher)
    14829    0.051    0.000    0.051    0.000 {built-in method _codecs.utf_8_decode}
    99977    0.045    0.000    0.045    0.000 fromnumeric.py:2612(_amax_dispatcher)
    99977    0.043    0.000    0.043    0.000 fromnumeric.py:1435(_s

---
# Evaluate the vector space

Verify if the trained model, or the vector space W, has encoded the words in a way that **similar** words are close in the vector space.

* [How to measure the similarity among vectors](https://math.stackexchange.com/questions/4132458)

In [34]:
n = 10
context = "cash".split()
word_indices = np.array(word_indexing.list_indices(context), dtype=TYPE_INT)

print(f"Words {context}")
print(f"Word indices {word_indices}")
print(f"prediction for {context}:\n{word_indexing.list_events([embedding.predict(word_indices, n)])}")

Words ['cash']
Word indices [413]
prediction for ['cash']:
[['interest' 'demand' 'fourth' 'debt' 'higher' 'lower' 'orders' 'tax' 'transactions' 'costs']
 ['demand' 'debt' 'fourth' 'orders' 'costs' 'assets' 'strong' 'drop' 'spending' 'fall']]


/home/oonisim/home/repository/git/oonisim/python_programs/nlp/src/layer/preprocessing/preprocess.py:216: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self._vocabulary[list(iter(indices))]


---
## Compare with [gensim word2vec](https://radimrehurek.com/gensim/models/word2vec.html)

In [38]:
from gensim.models import (
    Word2Vec
)
from gensim.models.word2vec import (
    LineSentence    
)

In [39]:
sentences = LineSentence(source=path_to_corpus)
w2v = Word2Vec(
    sentences=sentences, 
    sg=0,
    window=5, 
    negative=5,
    vector_size=100, 
    min_count=1, 
    workers=4
)
del sentences

In [40]:
w2v.wv.most_similar(context, topn=n)

[('amount', 0.8982115983963013),
 ('debt', 0.8907380104064941),
 ('assets', 0.8691324591636658),
 ('value', 0.8579981923103333),
 ('flow', 0.8463666439056396),
 ('face', 0.8402033448219299),
 ('payment', 0.8359115719795227),
 ('dividends', 0.8225480318069458),
 ('holders', 0.8186897039413452),
 ('payments', 0.8037282824516296)]